# Variables

## Copy on write

In [1]:
library(pryr)

Registered S3 method overwritten by 'pryr':
  method      from
  print.bytes Rcpp



In [2]:
x <- 158
address(x)

[1] "0x55c7ee3a0940"

In [3]:
y <- x
address(y)

[1] "0x55c7ee3a0940"

In [4]:
identical(address(x), address(y))

[1] TRUE

In [5]:
x <- 86
address(x)
address(y)

[1] "0x55c7eca34948"

[1] "0x55c7ee3a0940"

In [6]:
identical(address(x), address(y))

[1] FALSE

## Préallocation

In [7]:
lcg <- function(a, b, m,
                seed = 0)
  function(){
    ret <- (a * seed + b) %% m
    seed <<- ret
    ret / m
  }

In [8]:
## LCG de numerical recipes.
nr_lcg <- lcg(1664525, 1013904223, 2^32)

rand_prealloc <- function(n){
    res <- numeric(n)
    for (kk in 1:n)
        res[kk] <- nr_lcg()

    res
}

rand_vec <- function(n){
    replicate(n, nr_lcg())
}

In [9]:
library(microbenchmark)
print(microbenchmark(rand_vec(1e4), rand_prealloc(1e4)))

Unit: milliseconds
                 expr      min        lq     mean   median       uq      max
      rand_vec(10000) 14.01699 15.664319 18.66093 17.08496 20.33785 45.99623
 rand_prealloc(10000)  9.13592  9.896418 11.83521 10.77661 12.63599 27.69203
 neval
   100
   100


# Fonctions

## De niveau supérieur

In [10]:
Filter(function(x) identical(x %% 2, 0), 1:10)

[1]  2  4  6  8 10

## Composition

In [11]:
formals(lcg)

$a


$b


$m


$seed
[1] 0


In [12]:
body(lcg)

function() {
    ret <- (a * seed + b)%%m
    seed <<- ret
    ret/m
}

In [13]:
environment(lcg)

<environment: R_GlobalEnv>

## Portée lexicale

In [14]:
x <- 42
f <- function(){
    print(x)
    x <- 666
    print(x)
}
f()

[1] 42
[1] 666


In [15]:
print(x)

[1] 42


## Variable globale vs. closure

In [16]:
seed <- 42
lcg_bad <- function(a, b, m)
  function(){
    ret <- (a * seed + b) %% m
    seed <<- ret
    ret / m
  }

In [17]:
nr_lcg <- lcg(1664525, 1013904223, 2^32)

In [18]:
identical(environment(lcg), globalenv())

[1] TRUE

In [19]:
identical(environment(nr_lcg), globalenv())

[1] FALSE

# Comparaisons

## Approche impérative

In [20]:
add1 <- function(x) x + 1

add2 <- function(x) x + 2

for_add1_twice <- function(n){
    res <- integer(n)
    for (kk in 1:n)
        res[n] <- add1(add1(rpois(1, 1)))

    res
}

for_add2 <- function(n){
    res <- integer(n)
    for (kk in 1:n)
        res[n] <- add2(rpois(1, 1))

    res
}

for_noCall <- function(n){
    res <- integer(n)
    for (kk in 1:n)
        res[n] <- rpois(1, 1) + 2

    res
}

In [21]:
print(microbenchmark(for_add1_twice(1e4),
                     for_add2(1e4),
                     for_noCall(1e4)))

Unit: milliseconds
                  expr      min       lq     mean   median       uq      max
 for_add1_twice(10000) 22.72288 23.53105 27.56056 26.71269 28.07891 58.92085
       for_add2(10000) 18.98784 19.78932 23.86087 22.52685 24.55679 60.85150
     for_noCall(10000) 15.44881 15.94451 19.11916 19.14823 20.67933 28.52324
 neval
   100
   100
   100


## Approche fonctionnelle

In [22]:
sapply_add1_twice <- function(n)
    sapply(rpois(n, 1), function(x) add1(add1(x)))

sapply_add2 <- function(n) sapply(rpois(n, 1), add2)

sapply_noCall <- function(n) sapply(rpois(n, 1), `+`, y = 1)

In [23]:
print(microbenchmark(sapply_add1_twice(1e4),
                     sapply_add2(1e4),
                     sapply_noCall(1e4)))

Unit: milliseconds
                     expr       min        lq      mean    median        uq
 sapply_add1_twice(10000) 12.077698 13.695941 16.047801 14.333481 16.717061
       sapply_add2(10000)  5.927710  6.386837  7.227655  6.721923  7.454382
     sapply_noCall(10000)  3.601812  3.824616  4.632791  4.265314  4.819657
      max neval
 79.55702   100
 11.68015   100
 10.44032   100


## Approche array

In [24]:
add2_correct <- function(n) rpois(n, 1) + 2

In [25]:
print(microbenchmark(for_add1_twice(1e4),
                     for_add2(1e4),
                     for_noCall(1e4),
                     sapply_add1_twice(1e4),
                     sapply_add2(1e4),
                     sapply_noCall(1e4),
                     add2_correct(1e4)))

Unit: microseconds
                     expr       min         lq       mean    median        uq
    for_add1_twice(10000) 22746.755 24168.2605 27879.0273 27147.249 28979.543
          for_add2(10000) 19106.474 19909.2565 22930.0782 23375.155 24454.588
        for_noCall(10000) 15558.268 16231.5870 19161.3361 18101.639 20751.457
 sapply_add1_twice(10000) 12412.159 13307.8300 15012.7824 13809.200 15308.843
       sapply_add2(10000)  5779.844  6092.8375  7056.0104  6373.099  7079.894
     sapply_noCall(10000)  3532.019  3692.8440  4136.4587  3880.397  4200.979
      add2_correct(10000)   372.030   378.3925   418.6678   384.718   394.878
       max neval
 70330.037   100
 39338.760   100
 33932.068   100
 36772.695   100
 17223.236   100
  8135.214   100
  2071.860   100
